# **Tesing "One Stop Shop" Packages**

In [1]:
import itertools
import pandas as pd
import numpy as np
from random import gauss

from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import Ridge

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import warnings 
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Setup
data = 'data/'

class CFG:
    img_dim1 = 10
    img_dim2 = 6
    fontsize = 8
    marker = 1.5
    lines = 1.3

# plt.rcParams.keys() to list params
# adjust the parameters for displayed figures    
plt.rcParams.update({'figure.figsize': (CFG.img_dim1,CFG.img_dim2),
                     'font.size': (CFG.fontsize),
                     'lines.markersize': (CFG.marker),
                     'lines.linewidth': (CFG.lines)})  

In [3]:
# Func to calculate RMSE
def rmse(x, y):
    return (np.round(np.sqrt(mse(x, y)), 4))

In [4]:
# Get data from kaggle
from pathlib import Path
import zipfile
cred_path = Path('~/.kaggle/kaggle.json').expanduser()

def download_data(dataset, path, comp_check=True):
    os.makedirs(path, exist_ok=True)
    if any(Path(path).glob("*.zip")) or any(Path(path).iterdir()):
        print(f"Files already exist in '{path}'. Skipping download.")
        return
    if comp_check: kaggle_call = f"kaggle competitions download -c {dataset} -p {path}"
    else         : kaggle_call = f"kaggle datasets download -d {dataset} -p {path} --unzip"
    subprocess.run(kaggle_call, shell=True, check=True)
    for file in Path(path).glob("*.zip"):
        with zipfile.ZipFile(file, 'r') as zip_ref:
            zip_ref.extractall(path)
        print(f"Extracted {file}")
        file.unlink()
    print(f"Dataset '{dataset}' has been downloaded to '{path}'")